<center><h1>Learning and Applying the Fourier transform to the BraTS21' dataset</h1></center>

## Source of Learning:
 * [Prof. Sreeni Video](https://www.youtube.com/watch?v=9mLeVn8xzMw&ab_channel=Apeer_micro)
 * [Sreeni github](https://github.com/bnsreenu/python_for_image_processing_APEER/blob/master/tutorial41_image_filters_using_fourier_transform_DFT.py)

### This notebook acts as three things:
* a follow along as to how to apply a fourier transform to an image(video in sources)
* proof of concept that the principle works just fine on an mri image
* base for moving to next notebook which will aim to glean more value from the data using the transform more explicitly in our context

In [ ]:
import numpy as np 
import pandas as pd
import os
import glob
import re

import cv2

import matplotlib.pyplot as plt
import seaborn as sns
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

hidden cell for initial attempts at more graceful sorting(need to come back to)

In [ ]:
def tryint(s):
    try:
        return int(s)
    except ValueError:
        return s
    
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)
#source: https://nedbatchelder.com/blog/200712/human_sorting.html

In [ ]:
# flair_zero = sorted(glob_values, key= (lambda x:  alphanum_key(glob_values[x])))

In [ ]:
glob_values = glob.glob("../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/00001/FLAIR/*")

In [ ]:
glob_values[0]

## load image, resize, crop

In [ ]:
dicom = pydicom.read_file(glob_values[0])
data = dicom.pixel_array
data = cv2.resize(data, (256,256,)) #should edit this to be correct dimensions for dft shift

In [ ]:
plt.imshow(data)

In [ ]:
y = np.expand_dims(data, axis=-1)
plt.imshow(y)
y.shape

In [ ]:
from PIL import Image
im = Image.fromarray(data)
im.save("image.png")
img = cv2.imread('image.png',0)

In [ ]:
crop_img = img[40:160, 60:200]
plt.imshow(crop_img)

hidden cell below for testing the array values with different loading method

In [ ]:
# im = Image.fromarray(crop_img)
# im.save("image.png")
# img = cv2.imread('image.png',0)
# dft = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
# dft

## for filtering out low frequency

In [ ]:
dft = cv2.dft(np.float32(crop_img), flags=cv2.DFT_COMPLEX_OUTPUT)
dft.shape #comparing sizes

In [ ]:
dft_shift = np.fft.fftshift(dft)
magnitude_spectrum = 20 * np.log(cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]))
dft_shift.shape #comparing sizes

### resize image to work with script

In [ ]:
img = cv2.resize(data, (140,120,))

## FFT Processing Step (needs more customization for quality usage; acts as a proof of concept for now)

In [ ]:
rows, cols = img.shape
crow, ccol = int(rows / 2), int(cols / 2)

mask = np.ones((rows, cols, 2), np.uint8)
# r = 80
# center = [crow, ccol]
# x, y = np.ogrid[:rows, :cols]
# mask_area = (x - center[0]) ** 2 + (y - center[1]) ** 2 <= r*r
# mask[mask_area] = 0


# Circular LPF mask, center circle is 1, remaining all zeros
# Only allows low frequency components - smooth regions
#Can smooth out noise but blurs edges.
#
"""
rows, cols = img.shape
crow, ccol = int(rows / 2), int(cols / 2)
mask = np.zeros((rows, cols, 2), np.uint8)
r = 100
center = [crow, ccol]
x, y = np.ogrid[:rows, :cols]
mask_area = (x - center[0]) ** 2 + (y - center[1]) ** 2 <= r*r
mask[mask_area] = 1
# Band Pass Filter - Concentric circle mask, only the points living in concentric circle are ones
rows, cols = img.shape
crow, ccol = int(rows / 2), int(cols / 2)
mask = np.zeros((rows, cols, 2), np.uint8)
r_out = 80
r_in = 10
center = [crow, ccol]
x, y = np.ogrid[:rows, :cols]
mask_area = np.logical_and(((x - center[0]) ** 2 + (y - center[1]) ** 2 >= r_in ** 2),
                           ((x - center[0]) ** 2 + (y - center[1]) ** 2 <= r_out ** 2))
mask[mask_area] = 1
"""


# apply mask and inverse DFT: Multiply fourier transformed image (values)
#with the mask values. 
fshift = dft_shift * mask #dft_shift

#Get the magnitude spectrum (only for plotting purposes)
fshift_mask_mag = 20 * np.log(cv2.magnitude(fshift[:, :, 0], fshift[:, :, 1]))

#Inverse shift to shift origin back to top left.
f_ishift = np.fft.ifftshift(fshift)

#Inverse DFT to convert back to image domain from the frequency domain. 
#Will be complex numbers
img_back = cv2.idft(f_ishift)

#Magnitude spectrum of the image domain
img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])

## plotting step

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(2,2,1)
ax1.imshow(img, cmap='gray')
ax1.title.set_text('Input Image')
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(magnitude_spectrum, cmap='gray')
ax2.title.set_text('FFT of image')
ax3 = fig.add_subplot(2,2,3)
ax3.imshow(fshift_mask_mag, cmap='gray')
ax3.title.set_text('FFT + Mask')
ax4 = fig.add_subplot(2,2,4)
ax4.imshow(img_back, cmap='gray')
ax4.title.set_text('After inverse FFT')
plt.show()